In [1]:
import os
git_dir = os.path.dirname(os.getcwd())
project_dir = os.path.join(git_dir,'zavmo')
os.chdir(project_dir)

In [2]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [3]:
import json
from helpers.chat import get_openai_completion,  get_prompt, make_structured_call
from helpers.functions import parse_type
from IPython.display import Markdown

In [4]:
import openai
from openai import OpenAI
from helpers.functions import create_pydantic_model, get_yaml_data, create_system_message
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [44]:
from helpers.chat import create_message_payload, force_tool_call

In [92]:
get_yaml_data('profile')['primary']['fields']

[{'field': 'first_name',
  'title': 'first_name',
  'annotation': 'str',
  'description': "The first name of the user, often used to address them directly, e.g., 'Diana' or 'John'.",
  'user_examples': ["Hi, I'm Zavmo 😊, what's your name?",
   "Welcome to Zavmo, I'm Zavmo 😊, what's your name?",
   'What should I call you? Your first name would be awesome to know!',
   "Let's get to know each other! What's your first name? 🌟"],
  'probe_examples': ["It's wonderful to meet you, [first_name]! 🎉 You've just earned your first [X] XP. How about your last name? 😄",
   "Nice to meet you, [first_name]! You've unlocked a new learning journey. 🚀 Now, could you share your last name with me too?",
   "Cool name, [first_name]! Let's get started with some fun. 🎯 Oh, and what's your last name so I can address you properly when needed?",
   "Yay, [first_name]! You're on board. Time to learn and have fun. 💡🎉 But before we dive in, what's your last name?"]},
 {'field': 'last_name',
  'title': 'last_name'

In [94]:
primary_config = get_yaml_data('profile')['primary']

In [97]:
primary_model = create_pydantic_model(
    name=primary_config['title'],
    description=primary_config['description'],
    fields=[f for f in primary_config['fields'] if f['title'] in ['first_name','last_name']]
)

In [100]:
message_payload =[ 
    {"role":"system","content":get_prompt("profile/extract")},
    {"role":"user","content":f"Extract the user's first name and last name from the learner's message?\nLearner's message: My name is aditya chhabra"}]

In [101]:
primary_tool = force_tool_call(primary_model,
                message_payload,
                model='gpt-4o-mini',
                tool_choice='required',
                
               )

In [109]:
primary_tool.dict()

{'first_name': 'aditya', 'last_name': 'chhabra'}

In [77]:
fields = get_yaml_data("profile")['required']

In [86]:
identification_tool = force_tool_call(identification_model, 
                                      message_payload, 
                                      model='gpt-4o-mini',
                                      tool_choice='required',
                                      parallel_tool_calls=False)


In [27]:
identify_model     = create_pydantic_model(name=identification_data['title'],
                                   description=identification_data['description'],
                                   fields=identification_data['fields']
                                   )

In [21]:
primary_model     = create_pydantic_model(name=primary_data['title'],
                                   description=primary_data['description'],
                                   fields=primary_data['fields']
                                   )